In [101]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from pprint import pprint
from scipy import stats
from sklearn.model_selection import GridSearchCV
import statsmodels.api as sm

In [102]:
# file paths
file_path = '../Data/'
output_file_path = file_path + 'Output/'

In [103]:
# import the freeflow result based on non-simplified graph
ff_df = pd.read_csv(
    output_file_path + 'result0226/' + "freeflow_OD3am_all_googlerouteapi_new_graph_new_turn_control_slight.csv")

In [104]:
# import the google api result
gg_df_result_all = pd.read_csv(output_file_path + 'googlerouteapi2024allresult.csv')
# merge the freeflow travel time and google travel time into one dataframe
df = ff_df.merge(gg_df_result_all, left_on=['oid', 'did'], right_on=['oid', 'did'])

In [32]:
df['diff'] = df['duration'] - df['travel_time']

# Random Forest Regression Model

In [33]:
# split train and test set
train1, test1 = train_test_split(df, test_size=0.2, random_state=123)

In [34]:
y = train1['duration']
x =train1[['signal_count', 'stop_count', 'crossing_count', 'give_way_count','mini_roundabout_count','left_count','slight_left_count','right_count','slight_right_count', 'u_count','travel_time']]
x_test =test1[['signal_count', 'stop_count', 'crossing_count', 'give_way_count','mini_roundabout_count','left_count','slight_left_count','right_count','slight_right_count', 'u_count','travel_time']]
y_test = test1['duration']

In [94]:
train1

,Unnamed: 0_x,oid,did,oy_x,ox_x,dy_x,dx_x,hour_of_day,uber_time,travel_time,...,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time,distance_y,duration,polyline,diff
26114,26114,122821260,122628274,33.909488,-117.930809,33.891176,-117.921876,3,144.50,264.2,...,2569.0,3,144.50,38.51,139.55,1.30,4207,292,"[(33.90949, -117.93081), (33.90948, -117.93088...",27.8
29082,29082,122947547,123101885,34.161247,-118.393182,34.087089,-118.344060,3,761.35,596.2,...,1133.0,3,761.35,228.24,735.21,1.28,12425,807,"[(34.16125, -118.39318), (34.16126, -118.39213...",210.8
33018,33018,122855842,122918156,34.042944,-118.264155,34.039275,-118.282834,3,326.62,121.1,...,1384.0,3,326.62,174.82,295.64,1.54,2445,263,"[(34.04297, -118.26416), (34.04271, -118.26442...",141.9
25301,25301,122574242,122415416,34.259558,-118.291349,34.229771,-118.266207,3,671.34,300.3,...,346.0,3,671.34,729.01,551.55,1.64,4918,388,"[(34.25957, -118.29136), (34.25954, -118.28855...",87.7
26803,26803,559225576,123081733,34.005375,-118.489611,34.020466,-118.438184,3,680.78,368.7,...,1579.0,3,680.78,283.68,632.88,1.44,7344,526,"[(34.00538, -118.4896), (34.00601, -118.48871)...",157.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7763,7763,4026483334,123435773,33.832846,-118.081792,33.942110,-118.085381,3,657.39,595.1,...,1943.0,3,657.39,186.27,636.29,1.28,15236,660,"[(33.83282, -118.0818), (33.83284, -118.08194)...",64.9
15377,15377,123085519,122641374,33.974743,-118.244051,33.907912,-118.373447,3,1106.13,883.7,...,414.0,3,1106.13,265.09,1077.87,1.25,21723,1206,"[(33.97474, -118.24406), (33.97471, -118.24405...",322.3
17730,17730,122988502,122610300,34.022837,-118.108430,34.080807,-118.280092,3,804.29,906.7,...,1164.0,3,804.29,266.71,767.69,1.34,26006,1230,"[(34.02284, -118.10843), (34.02277, -118.10898...",323.3
28030,28030,3694058978,123502715,33.918717,-118.142177,33.917173,-118.008850,3,901.67,701.6,...,547.0,3,901.67,152.37,888.92,1.18,12413,858,"[(33.91871, -118.14217), (33.91882, -118.14209...",156.4


In [35]:
# Fitting the default Random forest Regression to the dataset
regressor = RandomForestRegressor()
# Fit the regressor with x and y data
regressor.fit(x, y)

RandomForestRegressor()

In [36]:
# Predict the result
predictions = regressor.predict(x_test)

In [37]:
Errors=abs(predictions-y_test)
print('Average baseline error:', round(np.mean(Errors), 3), ' seconds')

Average baseline error: 78.235  seconds


In [38]:
test1['rf_predict_default'] = regressor.predict(x_test)

In [39]:
# Evaluating the default random forest model: mean square error and r-squared
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')
r2 = r2_score(y_test, predictions)
print(f'R-squared: {r2}')


Mean Squared Error: 12739.030438273692
R-squared: 0.9308845450420487


## Freeflow travel time prediction

In [40]:
# Evaluating the default freeflow model: mean square error and r-squared
mse = mean_squared_error(y_test, test1['travel_time'])
print(f'Mean Squared Error: {mse}')
r2 = r2_score(y_test, test1['travel_time'])
print(f'R-squared: {r2}')

Mean Squared Error: 48155.429286750485
R-squared: 0.7387333031366679


In [41]:
errors = abs(y_test - test1['travel_time'])
print('Average Error: {:0.3f} seconds.'.format(np.mean(errors)))


Average Error: 183.546 seconds.


In [42]:
errors = abs(y_test - test1['travel_time'])
print('Median Error: {:0.3f} seconds.'.format(np.median(errors)))

Median Error: 157.950 seconds.


In [43]:
mape = 100 * np.mean(errors / y_test )
accuracy = 100 - mape
print('Average Accuracy = {:0.2f}%.'.format(accuracy))

Average Accuracy = 78.87%.


In [44]:
mape = 100 * np.median(errors / y_test )
accuracy = 100 - mape
print('Average Accuracy = {:0.2f}%.'.format(accuracy))

Average Accuracy = 79.46%.


## Try modelling using the difference as the dependent variable

In [45]:
# try modelling using the difference
y = train1['diff']
x =train1[['signal_count', 'stop_count', 'crossing_count', 'give_way_count','mini_roundabout_count','left_count','slight_left_count','right_count','slight_right_count', 'u_count']]
y_test = test1['diff']
x_test =test1[['signal_count', 'stop_count', 'crossing_count', 'give_way_count','mini_roundabout_count','left_count','slight_left_count','right_count','slight_right_count', 'u_count']]

In [46]:
regressor = RandomForestRegressor(random_state=123)
regressor.fit(x, y)

RandomForestRegressor(random_state=123)

In [47]:
predictions = regressor.predict(x_test)

In [48]:
Errors=abs(predictions-y_test)
print('Average baseline error:', round(np.mean(Errors), 3), ' seconds')

Average baseline error: 82.896  seconds


In [49]:
# Evaluating the model
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')
 
r2 = r2_score(y_test, predictions)
print(f'R-squared: {r2}')

Mean Squared Error: 14224.132364381543
R-squared: 0.03701655145744276


### It is a lot worse than modelling the google travel time, so we abandon the model that use the difference as the dependent variable

## Hyper tuning of random forest regression

In [50]:
regressor = RandomForestRegressor(random_state=123)

In [51]:
print('Parameters currently in use:\n')
pprint(regressor.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 123,
 'verbose': 0,
 'warm_start': False}


In [52]:
# Create and randomized grid of hyper parameters
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = ['sqrt', 'log2', None]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, None]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['sqrt', 'log2', None],
 'min_samples_split': [2, 5, 10, None],
 'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]}


In [53]:
train_labels = train1['duration']
train_features =train1[['signal_count', 'stop_count', 'crossing_count', 'give_way_count','mini_roundabout_count','left_count','slight_left_count','right_count','slight_right_count', 'u_count','travel_time']]
test_features =test1[['signal_count', 'stop_count', 'crossing_count', 'give_way_count','mini_roundabout_count','left_count','slight_left_count','right_count','slight_right_count', 'u_count','travel_time']]
test_labels = test1['duration']

In [54]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor(random_state = 123)
# Random search of parameters, using 5 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = GridSearchCV(estimator=rf,param_grid=random_grid, scoring='neg_mean_absolute_error', 
                              cv = 5, verbose=2, n_jobs= -1,
                              return_train_score=True)

# Fit the random search model
rf_random.fit(train_features, train_labels);

Fitting 5 folds for each of 2880 candidates, totalling 14400 fits
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_split=2, n_estimators=300; total time=   9.3s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_split=2, n_estimators=600; total time=  19.1s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_split=2, n_estimators=900; total time=  30.5s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_split=5, n_estimators=500; total time=  16.9s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_split=5, n_estimators=800; total time=  27.0s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_split=10, n_estimators=200; total time=   6.6s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_split=10, n_estimators=300; total time=  10.1s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_split=10, n_estimators=500; total time=  16.7s
[CV

/opt/miniconda3/envs/street-network-resilience/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_split=2, n_estimators=100; total time=   3.0s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_split=2, n_estimators=400; total time=  12.5s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_split=2, n_estimators=700; total time=  23.0s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_split=2, n_estimators=1000; total time=  33.7s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_split=5, n_estimators=800; total time=  26.9s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_split=10, n_estimators=100; total time=   3.5s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_split=10, n_estimators=200; total time=   6.7s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_split=10, n_estimators=300; total time=  10.1s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_sample

/opt/miniconda3/envs/street-network-resilience/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
3600 fits failed out of a total of 14400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1800 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/miniconda3/envs/street-network-resilience/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/miniconda3/envs/street-network-resilience/lib/python3.9/site-packages/sklearn/ensemble/_forest.py", line 476, in fit
    trees = Parallel(
  File "/opt/miniconda3/envs/street-network-resilience/lib/python3.9/site-packages/jobl

[CV] END bootstrap=False, max_depth=None, max_features=None, min_samples_split=10, n_estimators=700; total time= 2.7min
[CV] END bootstrap=False, max_depth=None, max_features=None, min_samples_split=10, n_estimators=1000; total time= 2.8min


In [68]:
rf_random.cv_results_

{'mean_fit_time': array([2.93850641, 5.92279329, 8.94268141, ..., 0.58405318, 0.67160635,
        0.69543033]),
 'std_fit_time': array([0.02010349, 0.02402511, 0.01872077, ..., 0.01130974, 0.02255881,
        0.00796965]),
 'mean_score_time': array([0.11746116, 0.24074283, 0.34318981, ..., 0.        , 0.        ,
        0.        ]),
 'std_score_time': array([0.00536012, 0.00389038, 0.00429285, ..., 0.        , 0.        ,
        0.        ]),
 'param_bootstrap': masked_array(data=[True, True, True, ..., False, False, False],
              mask=[False, False, False, ..., False, False, False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[10, 10, 10, ..., None, None, None],
              mask=[False, False, False, ..., False, False, False],
        fill_value='?',
             dtype=object),
 'param_max_features': masked_array(data=['sqrt', 'sqrt', 'sqrt', ..., None, None, None],
              mask=[False, False, False, ..., False, False, F

In [70]:
param_accuracy = pd.concat([pd.DataFrame(rf_random.cv_results_["params"]),pd.DataFrame(rf_random.cv_results_["mean_test_score"], columns=["Accuracy"])],axis=1)

In [88]:
param_accuracy

,bootstrap,max_depth,max_features,min_samples_split,n_estimators,Accuracy
0,True,10.0,sqrt,2.0,100,-86.698999
1,True,10.0,sqrt,2.0,200,-86.514466
2,True,10.0,sqrt,2.0,300,-86.056462
3,True,10.0,sqrt,2.0,400,-86.227739
4,True,10.0,sqrt,2.0,500,-86.037778
...,...,...,...,...,...,...
2875,False,NaN,None,NaN,600,NaN
2876,False,NaN,None,NaN,700,NaN
2877,False,NaN,None,NaN,800,NaN
2878,False,NaN,None,NaN,900,NaN


In [100]:
param_accuracy[(param_accuracy['n_estimators'] == 600) & (param_accuracy['min_samples_split'] == 2.0) & (param_accuracy['max_depth'] == 10)]


,bootstrap,max_depth,max_features,min_samples_split,n_estimators,Accuracy
5,True,10.0,sqrt,2.0,600,-85.987725
45,True,10.0,log2,2.0,600,-85.987725
85,True,10.0,None,2.0,600,-74.301555
1445,False,10.0,sqrt,2.0,600,-86.243009
1485,False,10.0,log2,2.0,600,-86.243009
1525,False,10.0,None,2.0,600,-79.662919


In [91]:
param_accuracy[(param_accuracy['n_estimators'] == 100) & (param_accuracy['min_samples_split'] == 2.0) & (param_accuracy['max_depth'].isna())]

,bootstrap,max_depth,max_features,min_samples_split,n_estimators,Accuracy
1320,True,NaN,sqrt,2.0,100,-76.693671
1360,True,NaN,log2,2.0,100,-76.693671
1400,True,NaN,None,2.0,100,-77.561833
2760,False,NaN,sqrt,2.0,100,-77.423658
2800,False,NaN,log2,2.0,100,-77.423658
2840,False,NaN,None,2.0,100,-105.135822


In [87]:
param_accuracy[(param_accuracy['n_estimators'] == 1000) & (param_accuracy['min_samples_split'] == 5.0) & (param_accuracy['max_depth'] == 10.0) & (param_accuracy['bootstrap'] == True)]

,bootstrap,max_depth,max_features,min_samples_split,n_estimators,Accuracy
19,True,10.0,sqrt,5.0,1000,-85.592835
59,True,10.0,log2,5.0,1000,-85.592835
99,True,10.0,None,5.0,1000,-74.277856


In [86]:
param_accuracy['max_depth'].value_counts()

sqrt    960
log2    960
Name: max_features, dtype: int64

In [71]:
param_accuracy.to_csv(output_file_path + 'result0331/' + 'param_accuracy_completegridsearch_0514.csv')

In [24]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor(random_state = 123)
# Random search of parameters, using 5 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = GridSearchCV(estimator=rf, param_distributions=random_grid,
                              n_iter = 100, scoring='neg_mean_absolute_error', 
                              cv = 5, verbose=2, random_state=123, n_jobs= -1,
                              return_train_score=True)

# Fit the random search model
rf_random.fit(train_features, train_labels);

TypeError: __init__() got an unexpected keyword argument 'param_distributions'

In [61]:
# the best hyper parameters
rf_random.best_params_

{'bootstrap': True,
 'max_depth': 10,
 'max_features': None,
 'min_samples_split': 10,
 'n_estimators': 1000}

In [56]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} seconds.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [57]:
base_model = RandomForestRegressor(n_estimators = 100, random_state = 123)
base_model.fit(train_features, train_labels)
base_accuracy = evaluate(base_model, test_features, test_labels)

Model Performance
Average Error: 78.0959 seconds.
Accuracy = 91.22%.


In [67]:
best_random = RandomForestRegressor(n_estimators = 1000, random_state = 123, min_samples_split = 10, max_features = None, max_depth=10, bootstrap=True)
best_random.fit(train_features, train_labels)
random_accuracy = evaluate(best_random, test_features, test_labels)

Model Performance
Average Error: 75.2461 seconds.
Accuracy = 91.56%.


In [62]:
predictions = best_random.predict(test_features)
# Evaluating the model
mse = mean_squared_error(test_labels, predictions)
print(f'Mean Squared Error: {mse}')
r2 = r2_score(y_test, predictions)
print(f'R-squared: {r2}')

Mean Squared Error: 12146.054519169767
R-squared: -49.6847106611882


In [63]:
test1['best_rf_predict'] = best_random.predict(test_features)


In [66]:
# t-test
stats.ttest_rel(test1['best_rf_predict'], test_labels)

Ttest_relResult(statistic=0.3654547214215459, pvalue=0.7147813451175697)

In [65]:
# save to csv
test1.to_csv(output_file_path + 'result0331/' + 'test1_best_rf_predict0514.csv')


In [95]:
res = sm.OLS(endog=test1['duration'], exog=test1[['best_rf_predict']]).fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:               duration   R-squared (uncentered):                   0.989
Model:                            OLS   Adj. R-squared (uncentered):              0.989
Method:                 Least Squares   F-statistic:                          7.262e+05
Date:                Wed, 15 May 2024   Prob (F-statistic):                        0.00
Time:                        02:05:42   Log-Likelihood:                         -50635.
No. Observations:                8272   AIC:                                  1.013e+05
Df Residuals:                    8271   BIC:                                  1.013e+05
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                      coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------

In [97]:
r2_score(test1['duration'], test1['best_rf_predict'])

0.9341017286908798

In [96]:
response = "duration"
predictors = ["best_rf_predict"]
data = test1[[response] + predictors].dropna()
X = data[predictors]
y = data[response]
model1 = sm.OLS(y, sm.add_constant(X))
result1 = model1.fit()
print(result1.summary())

                            OLS Regression Results                            
Dep. Variable:               duration   R-squared:                       0.934
Model:                            OLS   Adj. R-squared:                  0.934
Method:                 Least Squares   F-statistic:                 1.172e+05
Date:                Wed, 15 May 2024   Prob (F-statistic):               0.00
Time:                        02:07:58   Log-Likelihood:                -50635.
No. Observations:                8272   AIC:                         1.013e+05
Df Residuals:                    8270   BIC:                         1.013e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.9537      3.014     